In [1]:
import numpy as np
import pandas as pd
import requests
import time
from bs4 import BeautifulSoup
from selenium import webdriver

In [4]:
# 코드를 통한 웹 요청이 차단된 사이트
# url = "https://news.naver.com/main/list.nhn?mode=LS2D&mid=shm&sid2=283&sid1=105&date=20210129&page=1"
# resp = requests.get(url)

In [2]:
# selenium을 사용해서 웹 요청

path = "D:\ssac-one2\dev-tools\chromedriver.exe"
browser = webdriver.Chrome(executable_path=path)

In [3]:
browser.get("https://news.naver.com/main/list.nhn?mode=LS2D&mid=shm&sid2=283&sid1=105&date=20210129&page=1")

In [5]:
browser.page_source[:500]

'<html lang="ko" data-useragent="Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/88.0.4324.104 Safari/537.36"><head>\n<meta charset="euc-kr">\n<meta http-equiv="X-UA-Compatible" content="IE=edge">\n<meta name="referrer" contents="always">\n<meta http-equiv="refresh" content="600">\n<meta name="viewport" content="width=1106">\n\n    \n    \n        \n            \n                \n                    \n                    \n                    \n                    \n     '

In [6]:
news_html = browser.page_source

In [7]:
browser.close()

In [8]:
news_html[:100]

'<html lang="ko" data-useragent="Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML,'

In [9]:
soup = BeautifulSoup(news_html, "html.parser")

In [10]:
dl_list = soup.select('#main_content .list_body ul li dl')

In [30]:
rows = []
for dl in dl_list:
    links = dl.select('dt a')
    link = links[1] if len(links) == 2 else links[0]
    span = dl.select_one('dd span:nth-child(1)')
    # print(link.get_text().strip(), " / ", link['href'], span.get_text())
    rows.append([link.get_text().strip(), span.get_text().strip(), link['href']])

rows[:5]

[['비트코인, ‘일론 머스크’ 한 방에 16% ‘껑충’',
  '[디지털데일리 박현영기자] 하락과 반등을 반복하며 횡보하던 비트코인(BTC) 가격이 ‘일론 머스크 효과’로 급등했다. 29일 오 …',
  'https://news.naver.com/main/read.nhn?mode=LS2D&mid=shm&sid1=105&sid2=283&oid=138&aid=0002098288'],
 ['아토리서치, 경기도 스마트홈 보안 시스템 개발 완료',
  '(지디넷코리아=김윤희 기자)네트워크 소프트웨어 전문 기업 아토리서치(대표 정재웅)는 ‘경기도형 차세대 스마트홈 보안 시스템’ 개 …',
  'https://news.naver.com/main/read.nhn?mode=LS2D&mid=shm&sid1=105&sid2=283&oid=092&aid=0002212343'],
 ["이스트시큐리티 '알약', 네이버 SW 다운 기록 1위",
  "(지디넷코리아=김윤희 기자)보안 기업 이스트시큐리티(대표 정상원)는 자사 백신 프로그램 ‘알약' 공개용이 네이버 소프트웨어가 제 …",
  'https://news.naver.com/main/read.nhn?mode=LS2D&mid=shm&sid1=105&sid2=283&oid=092&aid=0002212338'],
 ['구글 클라우드, 제로 트러스트 구현 위한 ‘비욘드코프 엔터프라이즈’ 출시',
  '[디지털데일리 이종현기자] 구글 클라우드는 구글 통합형 제로 트러트스 제품군의 정식 버전 ‘비욘드코프 엔터프라이즈’를 출시했다고 …',
  'https://news.naver.com/main/read.nhn?mode=LS2D&mid=shm&sid1=105&sid2=283&oid=138&aid=0002098280'],
 ['아토리서치, 경기도형 차세대 스마트홈 보안 시스템 개발',
  '[디지털데일리 백지영기자] 아토리서치(대표 정재웅)는 ‘경기도형 차세대 스마트홈 보안 시스템’ 개발 및 실증을 완료했다고 29일 …',
  'https://n

In [43]:
from bs4.element import Tag, NavigableString, Comment

path = "D:\ssac-one2\dev-tools\chromedriver.exe"
browser2 = webdriver.Chrome(executable_path=path)

time.sleep(3)

for row in rows:
    browser2.get(row[2])
    soup2 = BeautifulSoup(browser2.page_source, "html.parser")

    contents_div = soup2.select_one('div#articleBodyContents')
    contents = []
    for child in contents_div.children: # children : 1단계 하위 요소들의 목록 반환
        # print( type(child), " / ", child.name )
        if type(child) == Tag and child.name != 'script': # <script>를 제외한 모든 <>의 내용
            contents.append(child.get_text().strip())
        elif type(child) == NavigableString:
            contents.append(str(child).strip())

    contents = [ c for c in contents if len(c) > 0 ] # "" 항목은 제거
    row.append(' '.join(contents)) # [ 'a', 'b', 'c' ] -> 'a b c'
    time.sleep(1)
    # break

browser2.close()

In [44]:
rows[0]

['비트코인, ‘일론 머스크’ 한 방에 16% ‘껑충’',
 '[디지털데일리 박현영기자] 하락과 반등을 반복하며 횡보하던 비트코인(BTC) 가격이 ‘일론 머스크 효과’로 급등했다. 29일 오 …',
 'https://news.naver.com/main/read.nhn?mode=LS2D&mid=shm&sid1=105&sid2=283&oid=138&aid=0002098288',
 '출처=일론 머스크 트위터 [디지털데일리 박현영기자] 하락과 반등을 반복하며 횡보하던 비트코인(BTC) 가격이 ‘일론 머스크 효과’로 급등했다. 29일 오후 6시 40분 빗썸 기준 비트코인 가격은 전날 같은 시간보다 15.88% 오른 4034만 3000원이다. 지난 11일 크게 급락한 이후 약 18일 만에 4000만원을 다시 넘겼다. 이번 상승은 일론 머스크 테슬라 CEO가 트위터 프로필에 ‘#bitcoin’ 해시태그를 추가하면서 발생했다. 그동안 머스크 CEO는 “월급을 비트코인으로 받을 수 있다”고 밝힌 적이 있을 정도로 비트코인에 긍정적인 의견을 제시해왔다. 비트코인 외 가상자산 중에서는 ‘도지코인’의 팬으로 알려져있다. <박현영기자> hyun@ddaily.co.kr ▶ 네이버에서 디지털데일리 채널 구독하기 ▶ IT정보의 즐거운 업그레이드 [딜라이트닷넷] <저작권자 © 디지털데일리 무단전재-재배포금지>']